In [1]:
import os
import pandas as pd
import numpy as np
!pip install surprise
#!pip install scikit-surprise # if the first line does not work

# Create Dataframe

In [2]:
# load data
df = pd.read_csv('./data/final_amazon_beauty.csv')
data = df[['rating', 'reviewer_id', 'product_id']]

/Users/bea/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Using Surprise

In [3]:
# from surprise import SVD, Reader
# from surprise import Dataset
# from surprise.model_selection import cross_validate

# # # Load the dataset (download it if needed)
# # data = Dataset.load_builtin('ml-100k')

# # As we're loading a custom dataset, we need to define a reader.
# reader = Reader(rating_scale=(0.5, 5))

# # Use the famous SVD algorithm
# algo = SVD()

# # Run 5-fold cross-validation and then print results
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [4]:
# from surprise import Reader, Dataset, SVD, evaluate

# Following Surprise documentation examples 
# https://surprise.readthedocs.io/en/stable/getting_started.html

from surprise import Reader, Dataset, SVD, NormalPredictor
from collections import defaultdict


from surprise import Dataset
from surprise.model_selection import cross_validate


# As we're loading a custom dataset, we need to define a reader.
reader = Reader(rating_scale=(0.5, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(data[['reviewer_id', 'product_id', 'rating']], reader)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2492  1.2545  1.2506  1.2597  1.2595  1.2547  0.0044  
MAE (testset)     0.9694  0.9707  0.9702  0.9765  0.9749  0.9723  0.0028  
Fit time          20.76   18.00   17.60   20.91   17.54   18.96   1.54    
Test time         0.61    0.57    0.57    0.62    0.57    0.59    0.02    


{'test_rmse': array([1.24915503, 1.25449262, 1.25056818, 1.25974427, 1.25945471]),
 'test_mae': array([0.96942168, 0.9706695 , 0.97015962, 0.97652494, 0.97490023]),
 'fit_time': (20.75985813140869,
  18.00378680229187,
  17.60459327697754,
  20.90755581855774,
  17.543304920196533),
 'test_time': (0.6109521389007568,
  0.5674901008605957,
  0.5658318996429443,
  0.6197917461395264,
  0.5749969482421875)}

In [5]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.2924536527485295
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [6]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
trainset = data.build_full_trainset()
algo.fit(trainset)

In [7]:
algo.predict(621,1)

Prediction(uid=621, iid=1, r_ui=None, est=4.112092528511223, details={'was_impossible': False})

,rating,verified,review_time,reviewer_id,product_id,reviewer_name,reviewer_text,summary,vote,title,brand,rank,main_cat,description,also_view,also_buy,price,similar_item,details
0,5,False,2000-06-03,A2XMFX1BR0IJFJ,0061073717,Jonathan Reed (jonathan.reed2@virgin.net),This calender is brilliant and has plenty of g...,Futurama rules,2,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
1,5,False,2000-05-06,ATKPYXA8XFKGJ,0061073717,Gwen Bates,This calender really is great. In addition to...,A great gift for any futurama fan!,NaN,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
2,1,True,2015-02-19,A1V6B6TNIC10QE,0143026860,theodore j bigham,great,One Star,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN


In [13]:
df_products = df[['product_id', 'title', 'brand']]
df_products.head(3)

,product_id,title,brand
0,0061073717,Workout Headphones by Arena Essentials,HarperCollins
1,0061073717,Workout Headphones by Arena Essentials,HarperCollins
2,0143026860,Black Diamond,Swedish Beauty


In [12]:
df_data = data.df
df_data

In [14]:
df_data = df_data.merge(df_products,how="left", on='product_id')
df_data

,reviewer_id,product_id,rating,title,brand
0,A2XMFX1BR0IJFJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
1,A2XMFX1BR0IJFJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
2,ATKPYXA8XFKGJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
3,ATKPYXA8XFKGJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
4,A1V6B6TNIC10QE,0143026860,1,Black Diamond,Swedish Beauty
...,...,...,...,...,...
313259234,A24KQ9RVU81L87,B01HJEGTYK,2,FESHFEN Scrunchy Scrunchies Synthetic Hair Bun...,NaN
313259235,A24KQ9RVU81L87,B01HJEGTYK,2,FESHFEN Scrunchy Scrunchies Synthetic Hair Bun...,NaN
313259236,A24KQ9RVU81L87,B01HJEGTYK,2,FESHFEN Scrunchy Scrunchies Synthetic Hair Bun...,NaN
313259237,A24KQ9RVU81L87,B01HJEGTYK,2,FESHFEN Scrunchy Scrunchies Synthetic Hair Bun...,NaN


In [15]:
test_reviewer_id = "ATKPYXA8XFKGJ"
df_data[df_data['reviewer_id']==test_reviewer_id].sort_values(by = 'rating',ascending=False)[:20]

,reviewer_id,product_id,rating,title,brand
2,ATKPYXA8XFKGJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
3,ATKPYXA8XFKGJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
